In [1]:
import os
import re
import string
import numpy as np

from keras.models import Sequential
from keras.layers import Dense, Dropout, LSTM
from keras.callbacks import ModelCheckpoint
from keras.utils import np_utils

Using TensorFlow backend.


In [2]:
def parse_observations(text):
    # Convert text to dataset. Treat each stanza as a sequence.
    
    line_counter = 0 
    lines = [line.strip() for line in text.split('\n')]

    obs_counter = 0
    obs = []
    obs_map = {}
    obs_elem = []

    for line in lines:
        # don't include sonnet numbers or blank lines
        if len(line) in [1, 2, 3]:
            line_counter = 0
            continue
        elif len(line) == 0:
            continue
  
        words = [word.strip() for word in line.split(' ')]
        for word in words:
            # make all words lowercase and remove punctuation
            word = re.sub(r'[^\w]', '', word).lower()
            
            if word not in obs_map:
                # Add unique words to the observations map.
                obs_map[word] = obs_counter
                obs_counter += 1
            
            # Add the encoded word.
            obs_elem.append(obs_map[word])
        
        line_counter += 1
        if line_counter % 4 == 0 or line_counter == 14:
            # Add the encoded sequence after end of each stanza
            obs.append(obs_elem)
            obs_elem = []
        
    return obs, obs_map

def parse_lines(text):
    # Convert text to dataset. Treat each line as a sequence.
    
    lines = [line.strip() for line in text.split('\n')]

    obs_counter = 0
    obs = []
    obs_map = {}

    for line in lines:
        # don't include sonnet numbers or blank lines
        if len(line) in [0, 1, 2, 3]:
            continue
  
        obs_elem = []
        words = [word.strip() for word in line.split(' ')]
        for word in words:
            # make all words lowercase and remove punctuation
            word = re.sub(r'[^\w]', '', word).lower()
            
            if word not in obs_map:
                # Add unique words to the observations map.
                obs_map[word] = obs_counter
                obs_counter += 1
            
            # Add the encoded word.
            obs_elem.append(obs_map[word])
        
        obs.append(obs_elem)
               
    return obs, obs_map

def parse_poems(text):
    # Convert text to dataset. Treat each poem as a sequence.
    
    line_counter = 0 
    lines = [line.strip() for line in text.split('\n')]

    obs_counter = 0
    obs = []
    obs_map = {}
    obs_elem = []

    for line in lines:
        # don't include sonnet numbers or blank lines
        if len(line) in [1, 2, 3]:
            line_counter = 0
            continue
        elif len(line) == 0:
            continue
  
        words = [word.strip() for word in line.split(' ')]
        for word in words:
            # make all words lowercase and remove punctuation
            word = re.sub(r'[^\w]', '', word).lower()
            
            if word not in obs_map:
                # Add unique words to the observations map.
                obs_map[word] = obs_counter
                obs_counter += 1
            
            # Add the encoded word.
            obs_elem.append(obs_map[word])
        
        line_counter += 1
        if line_counter == 14:
            # Add the encoded sequence after end of each stanza
            obs.append(obs_elem)
            obs_elem = []
        
    return obs, obs_map

In [29]:
text = open(os.path.join(os.getcwd(), 'data/shakespeare.txt')).read()

# get words as numbers and get the word map
all_words = []
words, word_map = parse_poems(text)
for i in range(len(words)):
    for j in range(len(words[i])):
        all_words.append(words[i][j])
words = all_words
        

# remove sonnet numbers and convert to lowercase
n_words = len(words)
n_vocab = len(word_map)

In [30]:
# prepare the dataset of input to output pairs encoded as integers
seq_length = 5
dataX = []
dataY = []

# use sliding window approach
for i in range(0, n_words - seq_length, 1):
    seq_in = words[i:i + seq_length]
    seq_out = words[i + seq_length]
    dataX.append([word for word in seq_in])
    dataY.append(seq_out)
    
n_sequences = len(dataX)
print(dataY)

[5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 10, 24, 21, 25, 13, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 34, 46, 34, 47, 28, 34, 48, 46, 49, 50, 26, 6, 51, 52, 15, 53, 54, 55, 56, 57, 58, 28, 15, 59, 60, 61, 29, 30, 62, 63, 34, 64, 56, 22, 65, 66, 67, 68, 69, 70, 15, 71, 72, 73, 74, 75, 76, 28, 77, 15, 53, 78, 18, 15, 79, 56, 80, 81, 82, 83, 84, 85, 34, 86, 56, 87, 88, 89, 68, 34, 8, 90, 34, 91, 92, 93, 94, 95, 96, 52, 97, 76, 41, 98, 99, 100, 101, 102, 103, 104, 105, 106, 43, 107, 34, 108, 45, 43, 107, 15, 109, 100, 34, 110, 111, 28, 112, 61, 29, 30, 88, 113, 32, 114, 115, 116, 117, 56, 118, 119, 120, 121, 122, 119, 123, 34, 8, 124, 125, 26, 126, 127, 74, 128, 129, 100, 130, 84, 131, 132, 133, 56, 134, 132, 135, 136, 137, 21, 108, 18, 138, 29, 74, 114, 28, 76, 139, 140, 81, 26, 51, 135, 56, 141, 34, 142, 143, 81, 26, 144, 145, 146, 147, 68, 34, 148, 56, 149, 15, 150, 26, 151, 52, 152, 15, 19, 6, 150, 17, 153, 154, 155,

In [31]:
# reshape X to be [samples, time steps, features]
X = np.reshape(dataX, (n_sequences, seq_length, 1))

# normalize data to range (0, 1)
X = X / float(n_vocab)

# one hot encode the output variable
y = np_utils.to_categorical(dataY)
print(X)
print(y)

[[[0.00000000e+00]
  [3.14861461e-04]
  [6.29722922e-04]
  [9.44584383e-04]
  [1.25944584e-03]]

 [[3.14861461e-04]
  [6.29722922e-04]
  [9.44584383e-04]
  [1.25944584e-03]
  [1.57430730e-03]]

 [[6.29722922e-04]
  [9.44584383e-04]
  [1.25944584e-03]
  [1.57430730e-03]
  [1.88916877e-03]]

 ...

 [[2.51889169e-01]
  [4.56863980e-01]
  [9.99370277e-01]
  [4.54659950e-01]
  [4.54659950e-01]]

 [[4.56863980e-01]
  [9.99370277e-01]
  [4.54659950e-01]
  [4.54659950e-01]
  [9.99685139e-01]]

 [[9.99370277e-01]
  [4.54659950e-01]
  [4.54659950e-01]
  [9.99685139e-01]
  [4.94332494e-02]]]
[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 1.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]


In [32]:
# define the LSTM model 
# (single layer with 150 units, followed by dense output layer)
model = Sequential()
model.add(LSTM(150, input_shape=(X.shape[1], X.shape[2])))
model.add(Dropout(0.2))
model.add(Dense(y.shape[1], activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam')

In [33]:
# define the checkpoint
filepath="weights-improvement-word-{epoch:02d}-{loss:.4f}.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='loss', verbose=1, save_best_only=True, mode='min')
callbacks_list = [checkpoint]

In [34]:
# train the model
model.fit(X, y, epochs=20, batch_size=128, callbacks=callbacks_list)

Epoch 1/20
17577/17577 [==============================] - 3s 148us/step - loss: 6.9521

Epoch 00001: loss improved from inf to 6.95213, saving model to weights-improvement-word-01-6.9521.hdf5
Epoch 2/20
17577/17577 [==============================] - 2s 131us/step - loss: 6.4650

Epoch 00002: loss improved from 6.95213 to 6.46499, saving model to weights-improvement-word-02-6.4650.hdf5
Epoch 3/20
17577/17577 [==============================] - 2s 130us/step - loss: 6.4428

Epoch 00003: loss improved from 6.46499 to 6.44284, saving model to weights-improvement-word-03-6.4428.hdf5
Epoch 4/20
17577/17577 [==============================] - 3s 144us/step - loss: 6.4361

Epoch 00004: loss improved from 6.44284 to 6.43607, saving model to weights-improvement-word-04-6.4361.hdf5
Epoch 5/20
17577/17577 [==============================] - 3s 149us/step - loss: 6.4331

Epoch 00005: loss improved from 6.43607 to 6.43310, saving model to weights-improvement-word-05-6.4331.hdf5
Epoch 6/20
17577/17577 [